In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from pandas.plotting import autocorrelation_plot
from plotting_functions import \
run_diagnostic_plots, plot_acf, plot_trace, plot_sensor_output, get_fname_list, plot_layer_posteriors
import visvis as vv

import geovis_notebook_version

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
author = 'rs' # 'dk' or 'rs

if author == 'dk': # user input - for david's results
    dir_parent = '/Volumes/david_hd/obsidian/output/experiments/'
    dir_output_diagnostics = 'output/plots-diagnostic'
    dir_output_sensors = 'output/plots-sensors'
    fname_samples = 'output.npz'

    dir_year = '2018'
    dir_month = '07'
    dir_day = '26'
    exp_str = '{}_{}_{}'.format(dir_month, dir_day, dir_year)
    num_list = ['01', '02', '03', '04', '05', '06']

    param_key_list = [
        'layer0rockProperties',
        'layer1rockProperties'
    ]
    param_idx_list = [0, 1]

    plot_function_list = [
        plot_acf,
        plot_trace
    ]

    plot_function_names = [
        'acf',
        'trace'
    ]
    # for sensor plot output
    sensor_name_list = ['grav', 'mag'] #'grav' or 'mag'

    data_names_list = [
        'magSensors','magReadings','gravSensors','gravReadings'
    ]
elif author == 'rs': # user input - for richard's results
    dir_parent = '/Volumes/david_hd/obsidian/output/experiments/'
    dir_output_diagnostics = 'output/plots-diagnostic'
    dir_output_sensors = 'output/plots-sensors'
    fname_samples = 'gascoyne_v4.npz'

    exp_str = 'gascoyne_v4_run'
    num_list = ['01', '02', '03', '04', '05', '06']
    #num_list = ['04']

    param_key_list = [
        'layer0rockProperties',
        'layer1rockProperties'
    ]
    param_idx_list = [0, 1]

    plot_function_list = [
        plot_acf,
        plot_trace
    ]

    plot_function_names = [
        'acf',
        'trace'
    ]
    # for sensor plot output
    sensor_name_list = ['grav', 'mag'] #'grav' or 'mag'

    data_names_list = [
        'magSensors','magReadings','gravSensors','gravReadings'
    ]

# Diagnostic plots

In [ ]:
# put together file paths
if author == 'dk':
    fpath_in_list = [
        os.path.join(dir_parent, exp_str, num, fname_samples)
        for num in num_list
    ]
    dir_out_list = [
        os.path.join(dir_parent, exp_str, num, dir_output_diagnostics)
        for num in num_list
    ]
elif author == 'rs':
    fpath_in_list = [
        os.path.join(dir_parent, exp_str + num, fname_samples)
        for num in num_list
    ]
    dir_out_list = [
        os.path.join(dir_parent, exp_str + num, dir_output_diagnostics)
        for num in num_list
    ]
print(fpath_in_list)
print(dir_out_list)

In [ ]:
# make plots
run_diagnostic_plots(
    fpath_in_list,
    dir_out_list,
    param_key_list,
    param_idx_list,
    plot_function_list,
    plot_function_names
)

# Sensor plots

In [ ]:
# put together file paths
if author == 'dk':  
    xp = lambda exp_str, num: os.path.join(exp_str, num)
    fname_samples = 'output.npz'
elif author == 'rs': 
    xp = lambda exp_str, num: exp_str + num
    fname_samples = 'gascoyne_v4-thin1000.npz'
    
fpath_samples_list = [
    os.path.join(dir_parent, xp(exp_str, num), fname_samples)
    for num in num_list
]
fpath_csv_list = [
    [
        os.path.join(dir_parent, xp(exp_str, num), "{}.csv".format(data_name))
        for data_name in data_names_list
    ]
    for num in num_list
]
dir_out_list = [
    os.path.join(dir_parent, xp(exp_str, num), dir_output_sensors)
    for num in num_list
]
print(dir_out_list)

In [ ]:
_ = plot_sensor_output(fpath_samples_list, dir_out_list, fpath_csv_list, data_names_list)

# Voxel plots

In [ ]:
# put together file paths
dir_output_voxels = 'output/plots-voxels'
search_str = 'voxel'
if author == 'dk':
    fpath_in_list = [
        get_fname_list(os.path.join(dir_parent, exp_str, num), search_str)
        for num in num_list
    ]
    dir_out_list = [
        os.path.join(dir_parent, exp_str, num, dir_output_voxels)
        for num in num_list
    ]
elif author == 'rs':
    fpath_in_list = [
        get_fname_list(os.path.join(dir_parent, exp_str + num), search_str)
        for num in num_list
    ]
    dir_out_list = [
        os.path.join(dir_parent, exp_str + num, dir_output_voxels)
        for num in num_list
    ]
print(fpath_in_list)
print(dir_out_list)

In [ ]:
fname_save_template_str = 'posterior-layer-{:02}.{}'
for voxel_path_list, dir_out in zip(fpath_in_list, dir_out_list):
    if voxel_path_list:
        plot_layer_posteriors(voxel_path_list, dir_out)

In [ ]:
# manually run visvis so you can move the plot around
fpath_idx = 6
layer_idx = 0
fn = fpath_in_list[fpath_idx][0]
print('filename: {}'.format(fn)
n_layers = len(view.layers)
print('no layers: {}'.format(n_layers)
view = geovis_notebook_version.MasonView(fn)
layer_voxels = view.meanlayer(layer_idx)

app = vv.use()
vv.figure(1)
vv.xlabel('Eastings (units)')
vv.ylabel('Northings (units)')
vv.zlabel('Depth (units)')
a = vv.gca()
print(a.GetView())
#a.loc = (0, 0, 100)
#a.azimuth = 100
#a.camera.fov = 10
a.daspect = 1, 1, -1
a.camera.elevation = -90
#a.camera.azimuth = 30
a.camera.roll = 10
print(a.GetView())
print(a.camera)
#a.camera.
a.SetView()
t = vv.volshow(layer_voxels, cm=vv.CM_JET)
#t = vv.volshow(layer_voxels, cm=vv.CM_JET, renderStyle='ray')
vv.ColormapEditor(a)
app.Run()